*This code calculates land-cover of crops in 1km buffers around the schisto sdm analysis feature collection -- data is downloaded for each year so much be subset to collection years in next processing steps*
> Original code written by João Siqueira



In [ ]:
# this code will create rasters that is distance to urban edges in Brazil
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=XlcV9lqYH0gfb6wUEJgcP5Mtndb_wAhJgbW9WtnqDkQ&tc=XKK8t_wvUw_YFLo5gNr4P17tIpbD0447disfZ2h_Mss&cc=I5aSGIrlixGjmSV4xMoAZPCiwKPWG7cp08ne2p7MAUA

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkYK6exYOaOHufZTpXx8fE3VwxmJaV0k2Koibtu6phoJp1yEt7C0AI

Successfully saved authorization token.


**READ IN DATA AND SET UP CODE**

In [ ]:
#Brazil feature for reducing images
region = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0').filter(ee.Filter.equals('ADM0_NAME', 'Brazil'));

# Read in snail points
all_sdm_points = ee.FeatureCollection('users/cglidden/all_points_schisto_sdm')

def buffer_points(radius, bounds):
    def buffer_feature(pt):
        pt = ee.Feature(pt)
        return ee.Algorithms.If(bounds, pt.buffer(radius).bounds(), pt.buffer(radius))
    return buffer_feature

# Asset of regions for which you want to calculate statistics - radius of 1km2
bufferedPoints = all_sdm_points.map(buffer_points(500, True))

# Asset mapbiomas
asset = 'projects/mapbiomas-workspace/public/collection7/mapbiomas_collection70_integration_v2'

# Numeric attribute to index the shapefile
attribute = "row_code"

# A list of class ids you are interested
classIds = [41, 21] # temp crops, mosaics of use

# Output csv name
outputName = 'all_points_sdm_lulc_crops'

# Change the scale if you need.
scale = 30

# Define a list of years to export
years = [
    '1985', '1986', '1987', '1988', '1989',
    '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
    '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
    '2019', '2020'
]

# Define a Google Drive output folder
driveFolder = 'final_schisto_data'

# Territory
territory = ee.FeatureCollection(bufferedPoints)

mapbiomas = ee.Image(asset)

# Image area in km2
pixelArea = ee.Image.pixelArea().divide(1000000);

# Geometry to export
geometry = mapbiomas.geometry();


**SET UP FUNCTIONS FOR GETTING LAND-USE COVER PER FEATURE PER YEAR**

In [ ]:
def convert2table(obj):
    obj = ee.Dictionary(obj)
    territory = obj.get('territory')
    classesAndAreas = ee.List(obj.get('groups'))

    def mapFunction(classAndArea):
        classAndArea = ee.Dictionary(classAndArea)
        classId = classAndArea.get('class')
        area = classAndArea.get('sum')

        tableColumns = ee.Feature(None)\
            .set(attribute, territory)\
            .set('class', classId)\
            .set('area', area)

        return tableColumns

    tableRows = classesAndAreas.map(mapFunction)
    return ee.FeatureCollection(ee.List(tableRows))

def calculateArea(image, territory, geometry):
    reducer = ee.Reducer.sum().group(groupField=1, groupName='class').group(groupField=1, groupName='territory')

    territoriesData = pixelArea.addBands(territory).addBands(image)\
        .reduceRegion(
            reducer=reducer,
            geometry=geometry,
            scale=scale,
            maxPixels=1e12
        )

    territoriesData = ee.List(territoriesData.get('groups'))
    areas = territoriesData.map(convert2table)
    areas = ee.FeatureCollection(areas).flatten()

    return areas


**APPLY FUNCTIONS TO GET AREA PER YEAR**

In [ ]:
def process_year(year):
    image = mapbiomas.select('classification_' + year)

    def calculate_area(feature):
        return calculateArea(
            image.remap(classIds, classIds, 0),
            ee.Image().int64().paint({
                'featureCollection': ee.FeatureCollection(feature),
                'color': attribute
            }),
            feature.geometry()
        )

    areas = territory.map(calculate_area)
    areas = areas.flatten()

    # Set additional properties
    areas = areas.map(lambda feature: feature.set('year', year))

    return areas

# Example usage:
year = '1985'  # Replace with the desired year
result = process_year(year)


EEException: ignored

In [ ]:
areas = ee.List([])

for year in years:
    image = mapbiomas.select('classification_' + year)

    def calculate_area(feature):
      return calculateArea(
        image.remap(classIds, classIds, 0),
        ee.Image().int64().paint(
            featureCollection=ee.FeatureCollection(feature),
            color=attribute
        ),
        feature.geometry()
    )

    areas_year = territory.map(calculate_area)
    areas_year = areas_year.flatten()

    # Set additional properties
    areas_year = areas_year.map(lambda feature: feature.set('year', year))

    areas = areas.add(areas_year)

areas = ee.FeatureCollection(areas).flatten()


**EXPORT TABLE**

In [ ]:
# Export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=areas,
    description=outputName,
    folder=driveFolder,
    fileNamePrefix=outputName,
    fileFormat='CSV'
)

# Start the export task
task.start()
